# Abrindo os bancos de dados de features e padronizando 
Os bancos estão em formato de container .h5 
São abertos, selecionados os IDs e exportados para csv em uma pasta, após isso é feita a leitura do csv e modificadas as colunas em ordem alfabética e o index renomeado sem o b''

In [1]:
import numpy as np
import pandas as pd
import networkx as nx
import sys, os, h5py
sys.path.append(os.path.abspath('../EMOGI'))
import gcnPreprocessing
import gcnIO
import matplotlib.pyplot as plt
%matplotlib inline

## Load Existing Data

In [2]:
data = gcnIO.load_hdf_data('C:/Users/renan/Desktop/UFRGS/GNN/EMOGI_Multinet/MULTINET_multiomics.h5')
network, features, y_train, y_val, y_test, train_mask, val_mask, test_mask, node_names, feat_names = data
features_df = pd.DataFrame(features, index=node_names[:, 0], columns=feat_names)

# Mudando o index=node_names[:, 0] para zero o identificador dos genes passa para ensembl gene id

features_df

,b'MF: UCEC',b'MF: BLCA',b'MF: THCA',b'MF: KIRC',b'MF: READ',b'MF: LUAD',b'MF: ESCA',b'MF: LUSC',b'MF: BRCA',b'MF: COAD',...,b'CNA: ESCA',b'CNA: LUSC',b'CNA: BRCA',b'CNA: COAD',b'CNA: HNSC',b'CNA: KIRP',b'CNA: PRAD',b'CNA: LIHC',b'CNA: STAD',b'CNA: CESC'
b'ENSG00000148584',0.010251,0.033728,0.000000,0.012698,0.027762,0.117901,0.000000,0.000000,0.031240,0.000000,...,0.283951,0.000000,0.000000,0.000000,0.000000,0.082251,0.000000,0.000000,0.00000,0.000000
b'',0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000
b'ENSG00000175899',0.019196,0.036284,0.003601,0.012836,0.000000,0.122328,0.027028,0.109769,0.010608,0.016204,...,0.000000,0.000000,0.000000,0.174419,0.000000,0.000000,0.000000,0.000000,0.23913,0.000000
b'ENSG00000094914',0.003829,0.000000,0.003604,0.025661,0.000000,0.000000,0.000000,0.000000,0.005306,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.18323,0.000000
b'ENSG00000081760',0.011557,0.000000,0.003602,0.000000,0.028397,0.020504,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.178794,0.227126,0.000000,0.159091,0.000000,0.129129,0.175074,0.00000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
b'ENSG00000070476',0.011466,0.000000,0.003603,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.010855,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000
b'ENSG00000162378',0.000000,0.000000,0.000000,0.012818,0.000000,0.000000,0.000000,0.000000,0.000000,0.010832,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000
b'ENSG00000285443',0.022981,0.000000,0.000000,0.012815,0.000000,0.020495,0.000000,0.000000,0.000000,0.010835,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.100890,0.00000,0.264317
b'ENSG00000074755',0.041296,0.178783,0.003603,0.038417,0.028239,0.059756,0.027013,0.000000,0.010602,0.010820,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.670623,0.00000,0.000000


In [3]:
features_df.to_csv('C:/Users/renan/Desktop/UFRGS/GNN/data/networks/Networks_ENSG/MULTINET_features.tsv', sep='\t')

In [4]:
# Leitura e padronização dos bancos de dados de features

MULTINET_features = pd.read_csv('C:/Users/renan/Desktop/UFRGS/GNN/data/networks/Networks_ENSG/MULTINET_features.tsv', sep= '\t')

# Colocando as colunas em odem alfabética
sorted_column_MULTINET_features = MULTINET_features.sort_index(axis=1)

# Removendo o caracter b'' das colunas, que vem no formato container
sorted_column_MULTINET_features.columns = sorted_column_MULTINET_features.columns.str.replace("b'", "")
sorted_column_MULTINET_features.columns = sorted_column_MULTINET_features.columns.str.replace("'", "")

# Removendo o cacacter b'' do index
sorted_column_MULTINET_features['Unnamed: 0'] = sorted_column_MULTINET_features['Unnamed: 0'].str.replace("b'","")
sorted_column_MULTINET_features['Unnamed: 0'] = sorted_column_MULTINET_features['Unnamed: 0'].str.replace("'","")

In [5]:
# Remover linhas que não tinham tradução

# O pandas não reconhece valores em branco a não ser que seja NaN para utilizar o drop
# Então primeiro atribui as lacunas como NaN e depois remove

sorted_column_MULTINET_features['Unnamed: 0'].replace('', np.nan, inplace=True)

sorted_column_MULTINET_features.dropna(subset=['Unnamed: 0'], inplace=True)

In [6]:
# Salvando o arquivo modificado
sorted_column_MULTINET_features.to_csv('C:/Users/renan/Desktop/UFRGS/GNN/data/networks/Networks_ENSG/MULTINET_features_padrao.tsv.tsv', sep='\t')

sorted_column_MULTINET_features

,Unnamed: 0,CNA: BLCA,CNA: BRCA,CNA: CESC,CNA: COAD,CNA: ESCA,CNA: HNSC,CNA: KIRC,CNA: KIRP,CNA: LIHC,...,MF: KIRC,MF: KIRP,MF: LIHC,MF: LUAD,MF: LUSC,MF: PRAD,MF: READ,MF: STAD,MF: THCA,MF: UCEC
0,ENSG00000148584,0.0,0.000000,0.000000,0.000000,0.283951,0.000000,0.000000,0.082251,0.000000,...,0.012698,0.000000,0.024672,0.117901,0.000000,0.090341,0.027762,0.011448,0.000000,0.010251
2,ENSG00000175899,0.0,0.000000,0.000000,0.174419,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.012836,0.040877,0.000000,0.122328,0.109769,0.022773,0.000000,0.038998,0.003601,0.019196
3,ENSG00000094914,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.025661,0.040872,0.000000,0.000000,0.000000,0.000000,0.000000,0.012746,0.003604,0.003829
4,ENSG00000081760,0.0,0.227126,0.000000,0.000000,0.000000,0.159091,0.000000,0.000000,0.175074,...,0.000000,0.000000,0.000000,0.020504,0.000000,0.000000,0.028397,0.038616,0.003602,0.011557
5,ENSG00000114771,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.346237,0.437229,0.000000,...,0.012836,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.007729
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14393,ENSG00000070476,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.346237,0.000000,0.000000,...,0.000000,0.040907,0.000000,0.000000,0.000000,0.000000,0.000000,0.025721,0.003603,0.011466
14394,ENSG00000162378,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.012818,0.081662,0.025013,0.000000,0.000000,0.000000,0.000000,0.038673,0.000000,0.000000
14395,ENSG00000285443,0.0,0.000000,0.264317,0.000000,0.000000,0.000000,0.376344,0.000000,0.100890,...,0.012815,0.000000,0.000000,0.020495,0.000000,0.000000,0.000000,0.012873,0.000000,0.022981
14396,ENSG00000074755,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.670623,...,0.038417,0.204110,0.000000,0.059756,0.000000,0.068216,0.028239,0.050702,0.003603,0.041296


In [7]:
data2 = gcnIO.load_hdf_data('C:/Users/renan/Desktop/UFRGS/GNN/EMOGI_CPDB/CPDB_multiomics.h5')
network, features, y_train, y_val, y_test, train_mask, val_mask, test_mask, node_names, feat_names = data2
features_df2 = pd.DataFrame(features, index=node_names[:, 0], columns=feat_names)

features_df2

,b'MF: KIRC',b'MF: BRCA',b'MF: READ',b'MF: PRAD',b'MF: STAD',b'MF: HNSC',b'MF: LUAD',b'MF: THCA',b'MF: BLCA',b'MF: ESCA',...,b'CNA: LUAD',b'CNA: THCA',b'CNA: BLCA',b'CNA: ESCA',b'CNA: LIHC',b'CNA: UCEC',b'CNA: COAD',b'CNA: LUSC',b'CNA: CESC',b'CNA: KIRP'
b'ENSG00000167323',0.000000,0.005282,0.000000,0.000000,0.012993,0.000000,0.060833,0.003601,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.449339,0.000000
b'ENSG00000144935',0.000000,0.005302,0.000000,0.022758,0.000000,0.021187,0.060202,0.000000,0.000000,0.053853,...,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000
b'ENSG00000089250',0.063345,0.015765,0.055677,0.022690,0.048721,0.021051,0.038974,0.000000,0.088398,0.000000,...,0.352785,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.190265,0.000000,0.000000
b'ENSG00000058668',0.038215,0.014866,0.053870,0.045420,0.025206,0.016322,0.040063,0.007193,0.057544,0.000000,...,0.000000,0.322222,0.272727,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000
b'ENSG00000069431',0.012769,0.031122,0.055910,0.045414,0.187905,0.048539,0.058909,0.010787,0.090623,0.053580,...,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
b'ENSG00000158292',0.000000,0.010610,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.323607,0.000000,0.000000,0.0,0.597786,0.0,0.000000,0.504425,0.176211,0.271357
b'ENSG00000140287',0.000000,0.015932,0.000000,0.022764,0.026045,0.005501,0.040862,0.003603,0.030570,0.000000,...,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.308370,0.175879
b'ENSG00000183117',0.062828,0.061911,0.108880,0.157486,0.346369,0.085493,0.332902,0.010780,0.256366,0.105089,...,0.673740,0.000000,0.920455,0.0,0.000000,0.0,0.582822,0.000000,0.000000,0.000000
b'ENSG00000180828',0.000000,0.005291,0.000000,0.000000,0.025961,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.077778,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000


In [8]:
features_df2.to_csv('C:/Users/renan/Desktop/UFRGS/GNN/data/networks/Networks_ENSG/CPDB_features.tsv', sep='\t')

In [9]:
CPDB_features = pd.read_csv('C:/Users/renan/Desktop/UFRGS/GNN/data/networks/Networks_ENSG/CPDB_features.tsv', sep= '\t')

# Colocando as colunas em odem alfabética
sorted_column_CPDB_features = CPDB_features.sort_index(axis=1)

# Removendo o caracter b'' das colunas, que vem no formato container
sorted_column_CPDB_features.columns = sorted_column_CPDB_features.columns.str.replace("b'", "")
sorted_column_CPDB_features.columns = sorted_column_CPDB_features.columns.str.replace("'", "")

# Removendo o cacacter b'' do index
sorted_column_CPDB_features['Unnamed: 0'] = sorted_column_CPDB_features['Unnamed: 0'].str.replace("b'","")
sorted_column_CPDB_features['Unnamed: 0'] = sorted_column_CPDB_features['Unnamed: 0'].str.replace("'","")

In [10]:
sorted_column_CPDB_features['Unnamed: 0'].replace('', np.nan, inplace=True)

sorted_column_CPDB_features.dropna(subset=['Unnamed: 0'], inplace=True)

In [11]:
# Salvando o arquivo modificado
sorted_column_CPDB_features.to_csv('C:/Users/renan/Desktop/UFRGS/GNN/data/networks/Networks_ENSG/CPDB_features_padrao.tsv', sep='\t')

sorted_column_CPDB_features

,Unnamed: 0,CNA: BLCA,CNA: BRCA,CNA: CESC,CNA: COAD,CNA: ESCA,CNA: HNSC,CNA: KIRC,CNA: KIRP,CNA: LIHC,...,MF: KIRC,MF: KIRP,MF: LIHC,MF: LUAD,MF: LUSC,MF: PRAD,MF: READ,MF: STAD,MF: THCA,MF: UCEC
0,ENSG00000167323,0.000000,0.000000,0.449339,0.000000,0.0,0.428947,0.000000,0.000000,0.000000,...,0.000000,0.040940,0.000000,0.060833,0.000000,0.000000,0.000000,0.012993,0.003601,0.007459
1,ENSG00000144935,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.346237,0.000000,0.000000,...,0.000000,0.040655,0.024799,0.060202,0.054278,0.022758,0.000000,0.000000,0.000000,0.007545
2,ENSG00000089250,0.000000,0.000000,0.000000,0.000000,0.0,0.165789,0.000000,0.000000,0.000000,...,0.063345,0.080900,0.048884,0.038974,0.103072,0.022690,0.055677,0.048721,0.000000,0.028677
3,ENSG00000058668,0.272727,0.000000,0.000000,0.000000,0.0,0.000000,0.148387,0.000000,0.000000,...,0.038215,0.000000,0.074138,0.040063,0.054143,0.045420,0.053870,0.025206,0.007193,0.014863
4,ENSG00000069431,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,...,0.012769,0.040659,0.024758,0.058909,0.000000,0.045414,0.055910,0.187905,0.010787,0.029335
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13622,ENSG00000158292,0.000000,0.000000,0.176211,0.000000,0.0,0.250000,0.212903,0.271357,0.597786,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.007677
13623,ENSG00000140287,0.000000,0.000000,0.308370,0.000000,0.0,0.000000,0.000000,0.175879,0.000000,...,0.000000,0.000000,0.000000,0.040862,0.000000,0.022764,0.000000,0.026045,0.003603,0.007667
13624,ENSG00000183117,0.920455,0.708492,0.000000,0.582822,0.0,0.000000,0.000000,0.000000,0.000000,...,0.062828,0.158012,0.169735,0.332902,0.248689,0.157486,0.108880,0.346369,0.010780,0.046940
13625,ENSG00000180828,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.040962,0.025052,0.000000,0.000000,0.000000,0.000000,0.025961,0.000000,0.003847


In [12]:
data3 = gcnIO.load_hdf_data('C:/Users/renan/Desktop/UFRGS/GNN/EMOGI_IRefIndex/IREF_multiomics.h5')
network, features, y_train, y_val, y_test, train_mask, val_mask, test_mask, node_names, feat_names = data3
features_df3 = pd.DataFrame(features, index=node_names[:, 0], columns=feat_names)

features_df3

,b'MF: COAD',b'MF: BRCA',b'MF: UCEC',b'MF: STAD',b'MF: LUSC',b'MF: KIRC',b'MF: CESC',b'MF: LIHC',b'MF: PRAD',b'MF: HNSC',...,b'CNA: CESC',b'CNA: LIHC',b'CNA: PRAD',b'CNA: HNSC',b'CNA: LUAD',b'CNA: ESCA',b'CNA: THCA',b'CNA: KIRP',b'CNA: BLCA',b'CNA: READ'
b'ENSG00000178498',0.000000,0.005271,0.000000,0.012973,0.000000,0.000000,0.053435,0.025035,0.000000,0.016379,...,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.136054
b'ENSG00000078967',0.000000,0.000000,0.003881,0.000000,0.000000,0.000000,0.028203,0.000000,0.022751,0.000000,...,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.011111,0.000000,0.0,0.000000
b'ENSG00000175325',0.000000,0.005315,0.000000,0.000000,0.000000,0.000000,0.027997,0.000000,0.000000,0.000000,...,0.0,0.0,0.095238,0.000000,0.334218,0.0,0.000000,0.241206,0.0,0.000000
b'ENSG00000185247',0.005439,0.015874,0.011579,0.038889,0.000000,0.000000,0.000000,0.000000,0.000000,0.016281,...,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.044444,0.000000,0.0,0.251701
b'ENSG00000204599',0.005370,0.005298,0.003753,0.012212,0.054052,0.000000,0.053123,0.000000,0.000000,0.010124,...,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
b'ENSG00000099984',0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000
b'ENSG00000124780',0.000000,0.000000,0.003856,0.000000,0.000000,0.012835,0.028197,0.000000,0.022770,0.005505,...,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000
b'ENSG00000166948',0.010868,0.005309,0.019106,0.026077,0.054257,0.025661,0.000000,0.000000,0.045526,0.016412,...,0.0,0.0,0.000000,0.000000,0.344828,0.0,0.000000,0.000000,0.0,0.000000
b'ENSG00000144847',0.005415,0.005271,0.003582,0.012370,0.000000,0.012793,0.052725,0.024650,0.022722,0.010936,...,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000


In [13]:
features_df3.to_csv('C:/Users/renan/Desktop/UFRGS/GNN/data/networks/Networks_ENSG/IREF_features.tsv', sep='\t')

In [14]:
IREF_features = pd.read_csv('C:/Users/renan/Desktop/UFRGS/GNN/data/networks/Networks_ENSG/IREF_features.tsv', sep= '\t')

# Colocando as colunas em odem alfabética
sorted_column_IREF_features = IREF_features.sort_index(axis=1)

# Removendo o caracter b'' das colunas, que vem no formato container
sorted_column_IREF_features.columns = sorted_column_IREF_features.columns.str.replace("b'", "")
sorted_column_IREF_features.columns = sorted_column_IREF_features.columns.str.replace("'", "")

# Removendo o cacacter b'' do index
sorted_column_IREF_features['Unnamed: 0'] = sorted_column_IREF_features['Unnamed: 0'].str.replace("b'","")
sorted_column_IREF_features['Unnamed: 0'] = sorted_column_IREF_features['Unnamed: 0'].str.replace("'","")

In [15]:
sorted_column_IREF_features['Unnamed: 0'].replace('', np.nan, inplace=True)

sorted_column_IREF_features.dropna(subset=['Unnamed: 0'], inplace=True)

In [16]:
# Salvando o arquivo modificado
sorted_column_IREF_features.to_csv('C:/Users/renan/Desktop/UFRGS/GNN/data/networks/Networks_ENSG/IREF_features_padrao.tsv', sep='\t')

sorted_column_IREF_features

,Unnamed: 0,CNA: BLCA,CNA: BRCA,CNA: CESC,CNA: COAD,CNA: ESCA,CNA: HNSC,CNA: KIRC,CNA: KIRP,CNA: LIHC,...,MF: KIRC,MF: KIRP,MF: LIHC,MF: LUAD,MF: LUSC,MF: PRAD,MF: READ,MF: STAD,MF: THCA,MF: UCEC
0,ENSG00000178498,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,...,0.000000,0.000000,0.025035,0.040857,0.000000,0.000000,0.0,0.012973,0.000000,0.000000
1,ENSG00000078967,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.022751,0.0,0.000000,0.000000,0.003881
2,ENSG00000175325,0.0,0.0,0.0,0.0,0.0,0.000000,0.713978,0.241206,0.0,...,0.000000,0.000000,0.000000,0.020519,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
3,ENSG00000185247,0.0,0.0,0.0,0.0,0.0,0.000000,0.079570,0.000000,0.0,...,0.000000,0.000000,0.000000,0.081227,0.000000,0.000000,0.0,0.038889,0.003603,0.011579
4,ENSG00000204599,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,...,0.000000,0.040836,0.000000,0.020380,0.054052,0.000000,0.0,0.012212,0.003598,0.003753
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17008,ENSG00000099984,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
17009,ENSG00000124780,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,...,0.012835,0.000000,0.000000,0.040468,0.000000,0.022770,0.0,0.000000,0.000000,0.003856
17010,ENSG00000166948,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,...,0.025661,0.081913,0.000000,0.000000,0.054257,0.045526,0.0,0.026077,0.000000,0.019106
17011,ENSG00000144847,0.0,0.0,0.0,0.0,0.0,0.000000,0.346237,0.000000,0.0,...,0.012793,0.000000,0.024650,0.039651,0.000000,0.022722,0.0,0.012370,0.000000,0.003582


In [17]:
data4 = gcnIO.load_hdf_data('C:/Users/renan/Desktop/UFRGS/GNN/EMOGI_PCNet/PCNET_multiomics.h5')
network, features, y_train, y_val, y_test, train_mask, val_mask, test_mask, node_names, feat_names = data4
features_df4 = pd.DataFrame(features, index=node_names[:, 0], columns=feat_names)

features_df4

,b'MF: PRAD',b'MF: ESCA',b'MF: LUAD',b'MF: BLCA',b'MF: KIRC',b'MF: LUSC',b'MF: STAD',b'MF: LIHC',b'MF: CESC',b'MF: BRCA',...,b'CNA: STAD',b'CNA: LIHC',b'CNA: CESC',b'CNA: BRCA',b'CNA: HNSC',b'CNA: THCA',b'CNA: READ',b'CNA: COAD',b'CNA: UCEC',b'CNA: KIRP'
b'ENSG00000160714',0.0,0.027109,0.000000,0.030771,0.012837,0.0,0.012896,0.0,0.000000,0.000000,...,0.0,0.000000,0.696035,0.000000,0.0,0.000000,0.326531,0.0,0.000000,0.000000
b'ENSG00000013561',0.0,0.000000,0.000000,0.000000,0.025470,0.0,0.012068,0.0,0.000000,0.010583,...,0.0,0.000000,0.000000,0.000000,0.0,0.233333,0.000000,0.0,0.000000,0.241206
b'ENSG00000140367',0.0,0.000000,0.000000,0.030507,0.000000,0.0,0.013017,0.0,0.000000,0.000000,...,0.0,0.000000,0.334802,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000
b'ENSG00000143183',0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.010269,...,0.0,0.000000,0.000000,0.000000,0.0,0.322222,0.000000,0.0,0.000000,0.000000
b'ENSG00000130560',0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.013057,0.0,0.000000,0.000000,...,0.0,0.439114,0.000000,0.376426,0.0,0.311111,0.000000,0.0,0.515625,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
b'',0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,...,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000
b'ENSG00000182103',0.0,0.000000,0.061346,0.000000,0.000000,0.0,0.013027,0.0,0.000000,0.000000,...,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000
b'ENSG00000188624',0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.013023,0.0,0.028289,0.000000,...,0.0,0.000000,0.167401,0.000000,0.0,0.000000,0.244898,0.0,0.000000,0.095477
b'',0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,...,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000


In [18]:
features_df4.to_csv('C:/Users/renan/Desktop/UFRGS/GNN/data/networks/Networks_ENSG/PCNET_features.tsv', sep='\t')

In [19]:
PCNET_features = pd.read_csv('C:/Users/renan/Desktop/UFRGS/GNN/data/networks/Networks_ENSG/PCNET_features.tsv', sep= '\t')

# Colocando as colunas em odem alfabética
sorted_column_PCNET_features = PCNET_features.sort_index(axis=1)

# Removendo o caracter b'' das colunas, que vem no formato container
sorted_column_PCNET_features.columns = sorted_column_PCNET_features.columns.str.replace("b'", "")
sorted_column_PCNET_features.columns = sorted_column_PCNET_features.columns.str.replace("'", "")

# Removendo o cacacter b'' do index
sorted_column_PCNET_features['Unnamed: 0'] = sorted_column_PCNET_features['Unnamed: 0'].str.replace("b'","")
sorted_column_PCNET_features['Unnamed: 0'] = sorted_column_PCNET_features['Unnamed: 0'].str.replace("'","")

In [20]:
sorted_column_PCNET_features['Unnamed: 0'].replace('', np.nan, inplace=True)

sorted_column_PCNET_features.dropna(subset=['Unnamed: 0'], inplace=True)

In [21]:
# Salvando o arquivo modificado
sorted_column_PCNET_features.to_csv('C:/Users/renan/Desktop/UFRGS/GNN/data/networks/Networks_ENSG/PCNET_features_padrao.tsv', sep='\t')

sorted_column_PCNET_features

,Unnamed: 0,CNA: BLCA,CNA: BRCA,CNA: CESC,CNA: COAD,CNA: ESCA,CNA: HNSC,CNA: KIRC,CNA: KIRP,CNA: LIHC,...,MF: KIRC,MF: KIRP,MF: LIHC,MF: LUAD,MF: LUSC,MF: PRAD,MF: READ,MF: STAD,MF: THCA,MF: UCEC
0,ENSG00000160714,0.000000,0.000000,0.696035,0.0,0.641379,0.0,0.133333,0.000000,0.000000,...,0.012837,0.000000,0.0,0.000000,0.000000,0.0,0.00000,0.012896,0.000000,0.011571
1,ENSG00000013561,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.241206,0.000000,...,0.025470,0.000000,0.0,0.000000,0.000000,0.0,0.00000,0.012068,0.007107,0.007325
2,ENSG00000140367,0.000000,0.000000,0.334802,0.0,0.000000,0.0,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.00000,0.013017,0.003560,0.003769
3,ENSG00000143183,0.000000,0.000000,0.000000,0.0,0.641379,0.0,0.133333,0.000000,0.000000,...,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.00000,0.000000,0.000000,0.003732
4,ENSG00000130560,0.000000,0.376426,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.439114,...,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.00000,0.013057,0.000000,0.011592
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19773,ENSG00000269693,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.00000,0.000000,0.000000,0.000000
19774,ENSG00000134297,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,...,0.012842,0.040982,0.0,0.020527,0.054925,0.0,0.00000,0.013020,0.000000,0.015488
19775,ENSG00000204345,0.734848,0.000000,0.000000,0.0,0.000000,0.0,0.000000,1.000000,0.000000,...,0.012842,0.122938,0.0,0.061632,0.000000,0.0,0.00000,0.052157,0.000000,0.000000
19777,ENSG00000182103,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.0,0.061346,0.000000,0.0,0.00000,0.013027,0.000000,0.000000


In [22]:
data5 = gcnIO.load_hdf_data('C:/Users/renan/Desktop/UFRGS/GNN/EMOGI_STRINGdb/STRINGdb_multiomics.h5')
network, features, y_train, y_val, y_test, train_mask, val_mask, test_mask, node_names, feat_names = data5
features_df5 = pd.DataFrame(features, index=node_names[:, 0], columns=feat_names)

features_df5

,b'MF: KIRP',b'MF: BLCA',b'MF: THCA',b'MF: ESCA',b'MF: CESC',b'MF: COAD',b'MF: LIHC',b'MF: LUAD',b'MF: PRAD',b'MF: STAD',...,b'CNA: LIHC',b'CNA: LUAD',b'CNA: PRAD',b'CNA: STAD',b'CNA: HNSC',b'CNA: LUSC',b'CNA: UCEC',b'CNA: BRCA',b'CNA: READ',b'CNA: KIRC'
b'ENSG00000004059',0.040985,0.000000,0.000000,0.000000,0.000000,0.005425,0.000000,0.000000,0.000000,0.000000,...,0.125461,0.000000,0.000000,0.0,0.000000,0.000000,0.246094,0.0,0.000000,0.376344
b'ENSG00000173898',0.081138,0.030386,0.007194,0.026539,0.127813,0.010051,0.048677,0.039826,0.021047,0.035777,...,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000
b'ENSG00000079616',0.000000,0.000000,0.000000,0.000000,0.056297,0.015818,0.000000,0.020363,0.022747,0.000000,...,0.000000,0.442971,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000
b'ENSG00000104671',0.000000,0.000000,0.000000,0.000000,0.028143,0.000000,0.000000,0.000000,0.000000,0.013015,...,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.115646,0.000000
b'ENSG00000105829',0.040930,0.030753,0.000000,0.000000,0.028077,0.000000,0.050092,0.000000,0.022755,0.000000,...,0.000000,0.000000,0.343537,0.0,0.444737,0.584071,0.000000,0.0,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
b'ENSG00000092931',0.081267,0.000000,0.000000,0.000000,0.083049,0.000000,0.024685,0.000000,0.000000,0.013008,...,0.000000,0.000000,0.000000,0.0,0.000000,0.522124,0.000000,0.0,0.000000,0.000000
b'ENSG00000139155',0.040815,0.030516,0.000000,0.026980,0.053995,0.000000,0.024792,0.039038,0.045482,0.051008,...,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.136054,0.000000
b'ENSG00000147100',0.000000,0.000000,0.000000,0.000000,0.111113,0.000000,0.000000,0.000000,0.045522,0.000000,...,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000
b'ENSG00000258436',0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.013037,...,0.000000,0.000000,0.000000,0.0,0.197368,0.387168,0.000000,0.0,0.476190,0.000000


In [23]:
features_df5.to_csv('C:/Users/renan/Desktop/UFRGS/GNN/data/networks/Networks_ENSG/STRINGdb_features.tsv', sep='\t')

In [24]:
STRINGdb_features = pd.read_csv('C:/Users/renan/Desktop/UFRGS/GNN/data/networks/Networks_ENSG/STRINGdb_features.tsv', sep= '\t')

# Colocando as colunas em odem alfabética
sorted_column_STRINGdb_features = STRINGdb_features.sort_index(axis=1)

# Removendo o caracter b'' das colunas, que vem no formato container
sorted_column_STRINGdb_features.columns = sorted_column_STRINGdb_features.columns.str.replace("b'", "")
sorted_column_STRINGdb_features.columns = sorted_column_STRINGdb_features.columns.str.replace("'", "")

# Removendo o cacacter b'' do index
sorted_column_STRINGdb_features['Unnamed: 0'] = sorted_column_STRINGdb_features['Unnamed: 0'].str.replace("b'","")
sorted_column_STRINGdb_features['Unnamed: 0'] = sorted_column_STRINGdb_features['Unnamed: 0'].str.replace("'","")

In [25]:
sorted_column_STRINGdb_features['Unnamed: 0'].replace('', np.nan, inplace=True)

sorted_column_STRINGdb_features.dropna(subset=['Unnamed: 0'], inplace=True)

In [26]:
# Salvando o arquivo modificado
sorted_column_STRINGdb_features.to_csv('C:/Users/renan/Desktop/UFRGS/GNN/data/networks/Networks_ENSG/STRINGdb_features_padrao.tsv', sep='\t')

sorted_column_STRINGdb_features

,Unnamed: 0,CNA: BLCA,CNA: BRCA,CNA: CESC,CNA: COAD,CNA: ESCA,CNA: HNSC,CNA: KIRC,CNA: KIRP,CNA: LIHC,...,MF: KIRC,MF: KIRP,MF: LIHC,MF: LUAD,MF: LUSC,MF: PRAD,MF: READ,MF: STAD,MF: THCA,MF: UCEC
0,ENSG00000004059,0.000000,0.0,0.264317,0.0,0.000000,0.000000,0.376344,0.000000,0.125461,...,0.000000,0.040985,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.011607
1,ENSG00000173898,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.012766,0.081138,0.048677,0.039826,0.053086,0.021047,0.025923,0.035777,0.007194,0.060659
2,ENSG00000079616,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.012803,0.000000,0.000000,0.020363,0.000000,0.022747,0.000000,0.000000,0.000000,0.000000
3,ENSG00000104671,0.954545,0.0,0.000000,0.0,0.517241,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.013015,0.000000,0.003847
4,ENSG00000105829,0.000000,0.0,0.000000,0.0,0.786207,0.444737,0.000000,0.000000,0.000000,...,0.000000,0.040930,0.050092,0.000000,0.000000,0.022755,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13174,ENSG00000092931,0.734848,0.0,0.000000,0.0,0.000000,0.000000,0.000000,1.000000,0.000000,...,0.000000,0.081267,0.024685,0.000000,0.053600,0.000000,0.000000,0.013008,0.000000,0.007231
13175,ENSG00000139155,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.038355,0.040815,0.024792,0.039038,0.162867,0.045482,0.000000,0.051008,0.000000,0.014983
13176,ENSG00000147100,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.012828,0.000000,0.000000,0.000000,0.054732,0.045522,0.028346,0.000000,0.000000,0.023163
13177,ENSG00000258436,0.000000,0.0,0.000000,0.0,0.000000,0.197368,0.000000,0.276382,0.000000,...,0.012843,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.013037,0.000000,0.000000


In [27]:
data6 = gcnIO.load_hdf_data('C:/Users/renan/Desktop/UFRGS/GNN/EMOGI_IRefIndex_2015/IREF_2015_multiomics.h5')
network, features, y_train, y_val, y_test, train_mask, val_mask, test_mask, node_names, feat_names = data6
features_df6 = pd.DataFrame(features, index=node_names[:, 0], columns=feat_names)

features_df6

,b'MF: ESCA',b'MF: KIRP',b'MF: HNSC',b'MF: PRAD',b'MF: BLCA',b'MF: LUAD',b'MF: LUSC',b'MF: BRCA',b'MF: KIRC',b'MF: CESC',...,b'CNA: LUSC',b'CNA: BRCA',b'CNA: KIRC',b'CNA: CESC',b'CNA: UCEC',b'CNA: STAD',b'CNA: THCA',b'CNA: COAD',b'CNA: READ',b'CNA: LIHC'
b'ENSG00000121410',0.000000,0.040970,0.005403,0.022774,0.000000,0.000000,0.000000,0.010629,0.012838,0.000000,...,0.0,0.0,0.000000,0.167401,0.261719,0.000000,0.000000,0.000000,0.244898,0.000000
b'ENSG00000148584',0.000000,0.000000,0.005363,0.090341,0.033728,0.117901,0.000000,0.031240,0.012698,0.021465,...,0.0,0.0,0.000000,0.000000,0.195313,0.000000,0.000000,0.000000,0.000000,0.000000
b'ENSG00000175899',0.027028,0.040877,0.016204,0.022773,0.036284,0.122328,0.109769,0.010608,0.012836,0.000000,...,0.0,0.0,0.000000,0.000000,0.000000,0.249191,0.000000,0.184049,0.136054,0.000000
b'ENSG00000094914',0.000000,0.040872,0.016425,0.000000,0.000000,0.000000,0.000000,0.005306,0.025661,0.000000,...,0.0,0.0,0.000000,0.000000,0.000000,0.190939,0.000000,0.000000,0.136054,0.000000
b'ENSG00000109576',0.000000,0.000000,0.010827,0.045023,0.000000,0.040790,0.000000,0.000000,0.012796,0.000000,...,0.0,0.0,0.167742,0.000000,0.000000,0.000000,0.055556,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
b'ENSG00000070476',0.000000,0.040907,0.010966,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.027067,...,0.0,0.0,0.346237,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
b'ENSG00000162378',0.000000,0.081662,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.012818,0.000000,...,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
b'ENSG00000285443',0.000000,0.000000,0.000000,0.000000,0.000000,0.020495,0.000000,0.000000,0.012815,0.000000,...,0.0,0.0,0.376344,0.264317,0.246094,0.000000,0.011111,0.000000,0.000000,0.125461
b'ENSG00000074755',0.027013,0.204110,0.016445,0.068216,0.178783,0.059756,0.000000,0.010602,0.038417,0.000000,...,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.088889,0.000000,0.000000,0.833948


In [28]:
features_df6.to_csv('C:/Users/renan/Desktop/UFRGS/GNN/data/networks/Networks_ENSG/IREF_2015_features.tsv', sep='\t')

In [29]:
IREF_2015_features = pd.read_csv('C:/Users/renan/Desktop/UFRGS/GNN/data/networks/Networks_ENSG/IREF_2015_features.tsv', sep= '\t')

# Colocando as colunas em odem alfabética
sorted_column_IREF_2015_features = IREF_2015_features.sort_index(axis=1)

# Removendo o caracter b'' das colunas, que vem no formato container
sorted_column_IREF_2015_features.columns = sorted_column_IREF_2015_features.columns.str.replace("b'", "")
sorted_column_IREF_2015_features.columns = sorted_column_IREF_2015_features.columns.str.replace("'", "")

# Removendo o cacacter b'' do index
sorted_column_IREF_2015_features['Unnamed: 0'] = sorted_column_IREF_2015_features['Unnamed: 0'].str.replace("b'","")
sorted_column_IREF_2015_features['Unnamed: 0'] = sorted_column_IREF_2015_features['Unnamed: 0'].str.replace("'","")

In [30]:
sorted_column_IREF_2015_features['Unnamed: 0'].replace('', np.nan, inplace=True)

sorted_column_IREF_2015_features.dropna(subset=['Unnamed: 0'], inplace=True)

In [31]:
# Salvando o arquivo modificado
sorted_column_IREF_2015_features.to_csv('C:/Users/renan/Desktop/UFRGS/GNN/data/networks/Networks_ENSG/IREF_2015_features_padrao.tsv', sep='\t')

sorted_column_IREF_2015_features

,Unnamed: 0,CNA: BLCA,CNA: BRCA,CNA: CESC,CNA: COAD,CNA: ESCA,CNA: HNSC,CNA: KIRC,CNA: KIRP,CNA: LIHC,...,MF: KIRC,MF: KIRP,MF: LIHC,MF: LUAD,MF: LUSC,MF: PRAD,MF: READ,MF: STAD,MF: THCA,MF: UCEC
0,ENSG00000121410,0.000000,0.0,0.167401,0.000000,0.000000,0.255263,0.000000,0.095477,0.000000,...,0.012838,0.040970,0.050114,0.000000,0.000000,0.022774,0.028288,0.013054,0.000000,0.007712
1,ENSG00000148584,0.000000,0.0,0.000000,0.000000,0.317241,0.000000,0.000000,0.095477,0.000000,...,0.012698,0.000000,0.024672,0.117901,0.000000,0.090341,0.027762,0.011448,0.000000,0.010251
2,ENSG00000175899,0.000000,0.0,0.000000,0.184049,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.012836,0.040877,0.000000,0.122328,0.109769,0.022773,0.000000,0.038998,0.003601,0.019196
3,ENSG00000094914,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.025661,0.040872,0.000000,0.000000,0.000000,0.000000,0.000000,0.012746,0.003604,0.003829
4,ENSG00000109576,0.670455,0.0,0.000000,0.000000,0.682759,0.000000,0.167742,0.000000,0.000000,...,0.012796,0.000000,0.000000,0.040790,0.000000,0.045023,0.000000,0.012517,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12124,ENSG00000070476,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.346237,0.000000,0.000000,...,0.000000,0.040907,0.000000,0.000000,0.000000,0.000000,0.000000,0.025721,0.003603,0.011466
12125,ENSG00000162378,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.012818,0.081662,0.025013,0.000000,0.000000,0.000000,0.000000,0.038673,0.000000,0.000000
12126,ENSG00000285443,0.000000,0.0,0.264317,0.000000,0.000000,0.000000,0.376344,0.000000,0.125461,...,0.012815,0.000000,0.000000,0.020495,0.000000,0.000000,0.000000,0.012873,0.000000,0.022981
12127,ENSG00000074755,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.833948,...,0.038417,0.204110,0.000000,0.059756,0.000000,0.068216,0.028239,0.050702,0.003603,0.041296


# Processamento das features completas para todas as redes

In [32]:
teste_todas_redes = pd.concat([sorted_column_MULTINET_features, sorted_column_CPDB_features, sorted_column_IREF_features, 
                                sorted_column_PCNET_features, sorted_column_STRINGdb_features, sorted_column_IREF_2015_features])

teste_todas_redes

,Unnamed: 0,CNA: BLCA,CNA: BRCA,CNA: CESC,CNA: COAD,CNA: ESCA,CNA: HNSC,CNA: KIRC,CNA: KIRP,CNA: LIHC,...,MF: KIRC,MF: KIRP,MF: LIHC,MF: LUAD,MF: LUSC,MF: PRAD,MF: READ,MF: STAD,MF: THCA,MF: UCEC
0,ENSG00000148584,0.0,0.000000,0.000000,0.000000,0.283951,0.000000,0.000000,0.082251,0.000000,...,0.012698,0.000000,0.024672,0.117901,0.000000,0.090341,0.027762,0.011448,0.000000,0.010251
2,ENSG00000175899,0.0,0.000000,0.000000,0.174419,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.012836,0.040877,0.000000,0.122328,0.109769,0.022773,0.000000,0.038998,0.003601,0.019196
3,ENSG00000094914,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.025661,0.040872,0.000000,0.000000,0.000000,0.000000,0.000000,0.012746,0.003604,0.003829
4,ENSG00000081760,0.0,0.227126,0.000000,0.000000,0.000000,0.159091,0.000000,0.000000,0.175074,...,0.000000,0.000000,0.000000,0.020504,0.000000,0.000000,0.028397,0.038616,0.003602,0.011557
5,ENSG00000114771,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.346237,0.437229,0.000000,...,0.012836,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.007729
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12124,ENSG00000070476,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.346237,0.000000,0.000000,...,0.000000,0.040907,0.000000,0.000000,0.000000,0.000000,0.000000,0.025721,0.003603,0.011466
12125,ENSG00000162378,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.012818,0.081662,0.025013,0.000000,0.000000,0.000000,0.000000,0.038673,0.000000,0.000000
12126,ENSG00000285443,0.0,0.000000,0.264317,0.000000,0.000000,0.000000,0.376344,0.000000,0.125461,...,0.012815,0.000000,0.000000,0.020495,0.000000,0.000000,0.000000,0.012873,0.000000,0.022981
12127,ENSG00000074755,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.833948,...,0.038417,0.204110,0.000000,0.059756,0.000000,0.068216,0.028239,0.050702,0.003603,0.041296


In [33]:
# Removendo duplicadas a partir do nome dos genes e mantendo a primeira aparição

teste = teste_todas_redes.drop_duplicates(subset = ['Unnamed: 0'], keep = 'first')

teste

,Unnamed: 0,CNA: BLCA,CNA: BRCA,CNA: CESC,CNA: COAD,CNA: ESCA,CNA: HNSC,CNA: KIRC,CNA: KIRP,CNA: LIHC,...,MF: KIRC,MF: KIRP,MF: LIHC,MF: LUAD,MF: LUSC,MF: PRAD,MF: READ,MF: STAD,MF: THCA,MF: UCEC
0,ENSG00000148584,0.0,0.000000,0.000000,0.000000,0.283951,0.000000,0.000000,0.082251,0.000000,...,0.012698,0.000000,0.024672,0.117901,0.000000,0.090341,0.027762,0.011448,0.000000,0.010251
2,ENSG00000175899,0.0,0.000000,0.000000,0.174419,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.012836,0.040877,0.000000,0.122328,0.109769,0.022773,0.000000,0.038998,0.003601,0.019196
3,ENSG00000094914,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.025661,0.040872,0.000000,0.000000,0.000000,0.000000,0.000000,0.012746,0.003604,0.003829
4,ENSG00000081760,0.0,0.227126,0.000000,0.000000,0.000000,0.159091,0.000000,0.000000,0.175074,...,0.000000,0.000000,0.000000,0.020504,0.000000,0.000000,0.028397,0.038616,0.003602,0.011557
5,ENSG00000114771,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.346237,0.437229,0.000000,...,0.012836,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.007729
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13167,ENSG00000215910,0.0,0.000000,0.176211,0.000000,0.000000,0.250000,0.212903,0.271357,0.000000,...,0.000000,0.000000,0.025064,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.003806
13168,ENSG00000275356,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
13171,ENSG00000256222,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2873,ENSG00000184911,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [35]:
teste2 = teste.rename(columns={"Unnamed: 0": "gene"})

teste2

,gene,CNA: BLCA,CNA: BRCA,CNA: CESC,CNA: COAD,CNA: ESCA,CNA: HNSC,CNA: KIRC,CNA: KIRP,CNA: LIHC,...,MF: KIRC,MF: KIRP,MF: LIHC,MF: LUAD,MF: LUSC,MF: PRAD,MF: READ,MF: STAD,MF: THCA,MF: UCEC
0,ENSG00000148584,0.0,0.000000,0.000000,0.000000,0.283951,0.000000,0.000000,0.082251,0.000000,...,0.012698,0.000000,0.024672,0.117901,0.000000,0.090341,0.027762,0.011448,0.000000,0.010251
2,ENSG00000175899,0.0,0.000000,0.000000,0.174419,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.012836,0.040877,0.000000,0.122328,0.109769,0.022773,0.000000,0.038998,0.003601,0.019196
3,ENSG00000094914,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.025661,0.040872,0.000000,0.000000,0.000000,0.000000,0.000000,0.012746,0.003604,0.003829
4,ENSG00000081760,0.0,0.227126,0.000000,0.000000,0.000000,0.159091,0.000000,0.000000,0.175074,...,0.000000,0.000000,0.000000,0.020504,0.000000,0.000000,0.028397,0.038616,0.003602,0.011557
5,ENSG00000114771,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.346237,0.437229,0.000000,...,0.012836,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.007729
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13167,ENSG00000215910,0.0,0.000000,0.176211,0.000000,0.000000,0.250000,0.212903,0.271357,0.000000,...,0.000000,0.000000,0.025064,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.003806
13168,ENSG00000275356,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
13171,ENSG00000256222,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2873,ENSG00000184911,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [36]:
teste2.to_csv('C:/Users/renan/Desktop/UFRGS/GNN/data/unity_features_ensg.tsv', sep='\t')